### Testing Realization Parameters

In [1]:
import json
import fiona
import geopandas
import pandas as pd
from collections import OrderedDict

In [2]:
wb_id = 'wb-2917533'

In [3]:
gpkg = f'{wb_id}/{wb_id}_upstream_subset.gpkg'

In [4]:
fiona.listlayers(gpkg)

['flowpaths',
 'divides',
 'nexus',
 'flowpath_edge_list',
 'flowpath_attributes',
 'hydrolocations',
 'lakes']

In [5]:
flowpaths = geopandas.read_file(gpkg, layer='flowpaths')
#flowpaths.dtypes

In [6]:
divides = geopandas.read_file(gpkg, layer='divides')
#divides.dtypes

In [7]:
nexus = geopandas.read_file(gpkg, layer='nexus')
#nexus.dtypes

In [8]:
#crosswalk = geopandas.read_file(gpkg, layer='crosswalk')
#crosswalk.dtypes

In [9]:
flowpath_edge_list = geopandas.read_file(gpkg, layer='flowpath_edge_list')
#flowpath_edge_list.dtypes

In [10]:
flowpath_attributes = geopandas.read_file(gpkg, layer='flowpath_attributes')
#flowpath_attributes.dtypes

In [11]:
cfe_noahowp_attributes = pd.read_csv(f'{wb_id}/cfe_noahowp_attributes.csv')

In [12]:
cfe_noahowp_attributes

,divide_id,gw_Coeff,gw_Zmax,gw_Expon,bexp_soil_layers_stag=1,bexp_soil_layers_stag=2,bexp_soil_layers_stag=3,bexp_soil_layers_stag=4,ISLTYP,IVGTYP,...,slope,smcmax_soil_layers_stag=1,smcmax_soil_layers_stag=2,smcmax_soil_layers_stag=3,smcmax_soil_layers_stag=4,smcwlt_soil_layers_stag=1,smcwlt_soil_layers_stag=2,smcwlt_soil_layers_stag=3,smcwlt_soil_layers_stag=4,vcmx25
0,cat-2918021,0.005,122.649315,1.0,2.328283,2.328283,2.328283,2.328283,3,14,...,0.374807,0.520030,0.520030,0.520030,0.520030,0.047000,0.047000,0.047000,0.047000,60.817778
1,cat-2917496,0.005,122.649315,1.0,2.328283,2.328283,2.328283,2.328283,3,14,...,0.374807,0.520030,0.520030,0.520030,0.520030,0.047000,0.047000,0.047000,0.047000,67.157300
2,cat-2917946,0.005,122.649315,1.0,2.328283,2.328283,2.328283,2.328283,3,14,...,0.374807,0.520030,0.520030,0.520030,0.520030,0.047000,0.047000,0.047000,0.047000,59.262777
3,cat-2918020,0.005,122.649315,1.0,2.328283,2.328283,2.328283,2.328283,3,14,...,0.374807,0.520030,0.520030,0.520030,0.520030,0.047000,0.047000,0.047000,0.047000,68.969184
4,cat-2917497,0.005,122.649315,1.0,2.328283,2.328283,2.328283,2.328283,3,14,...,0.374807,0.520030,0.520030,0.520030,0.520030,0.047000,0.047000,0.047000,0.047000,69.760651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,cat-2917951,0.005,55.234650,2.0,2.358826,2.358826,2.358826,2.358826,6,11,...,0.165568,0.462224,0.462224,0.462224,0.462224,0.059996,0.059996,0.059996,0.059996,76.482964
246,cat-2918270,0.005,55.234650,2.0,2.358826,2.358826,2.358826,2.358826,6,11,...,0.160008,0.466133,0.466133,0.466133,0.466133,0.066000,0.066000,0.066000,0.066000,80.874229
247,cat-2918271,0.005,55.234650,2.0,2.358826,2.358826,2.358826,2.358826,6,11,...,0.165438,0.463937,0.463937,0.463937,0.463937,0.065968,0.065968,0.065968,0.065968,79.117785
248,cat-2917791,0.005,55.234650,2.0,2.358826,2.358826,2.358826,2.358826,6,1,...,0.165568,0.468165,0.468165,0.468165,0.468165,0.067249,0.067249,0.067249,0.067249,39.268737


In [13]:
# reference doc
# https://github.com/NOAA-OWP/ngen/blob/master/doc/REALIZATION_CONFIGURATION.md
catchment_configs = {}
for idx, row in cfe_noahowp_attributes.iterrows():
    d = OrderedDict()
    
    # static parameters
    d['forcing_file']='BMI'
    d['surface_partitioning_scheme']='Schaake'

    # ----------------    
    # State Parameters
    # ----------------

    # soil depth
    d['soil_params.depth']='2.0[m]'

    # many of these values are taken from the 2m depth in hydrofabrics cfe_noahowp_attributes
    d['soil_params.b']=f'{row["bexp_soil_layers_stag=2"]}[]' # 	beta exponent on Clapp-Hornberger (1978) soil water relations

    # saturated hydraulic conductivity
    d['soil_params.satdk']=f'{row["dksat_soil_layers_stag=2"]}[m s-1]' 
    
    # saturated capillary head
    d['soil_params.satpsi']=f'{row["psisat_soil_layers_stag=2"]}[m]'
    
    # this factor (0-1) modifies the gradient of the hydraulic head at the soil bottom. 0=no-flow.
    d['soil_params.slop']=f'{row["slope"]}[m/m]'
    
    # saturated soil moisture content
    d['soil_params.smcmax']=f'{row["smcmax_soil_layers_stag=2"]}[m/m]'
    
    # wilting point soil moisture content
    d['soil_params.wltsmc']=f'{row["smcwlt_soil_layers_stag=2"]}[m/m]'
    
    # ---------------------    
    # Adjustable Parameters
    # ---------------------
    
    # optional; defaults to 1.0
    d['soil_params.expon']=f'{row["gw_Expon"]}[]' if row["gw_Expon"] is not None else '1.0[]'
    
    # optional; defaults to 1.0 
    # not sure if this is the correct key
    d['soil_params.expon_secondary']=f'{row["gw_Coeff"]}[]' if row["gw_Coeff"] is not None else '1.0[]'
    
    # maximum storage in the conceptual reservoir
    d['max_gw_storage']=f'{row["gw_Zmax"]}[m]' if row["gw_Zmax"] is not None else '0.011[m]'

     # primary outlet coefficient
    d['Cgw']='0.0018[m h-1]'

    # exponent parameter (1.0 for linear reservoir)
    d['expon']='6.0[]' 

    # initial condition for groundwater reservoir - it is the ground water as a 
    # decimal fraction of the maximum groundwater storage (max_gw_storage) for the initial timestep
    d['gw_storage']='0.05[m/m]'

    # field capacity
    d['alpha_fc']='0.33' 
    
    # initial condition for soil reservoir - it is the water in the soil as a 
    # decimal fraction of maximum soil water storage (smcmax * depth) for the initial timestep
    d['soil_storage']='0.05[m/m]' 
    
    # number of Nash lf reservoirs (optional, defaults to 2, ignored if storage values present)
    d['K_nash']='0.03[]' 
    
    # Nash Config param - primary reservoir
    d['K_lf']='0.01[]' 
    
    # Nash Config param - secondary reservoir
    d['nash_storage']='0.0,0.0' 
    
    # Giuh ordinates in dt time steps
    d['giuh_ordinates']='1.00,0.00'
    
    # ---------------------    
    # Time Info
    # ---------------------
    
    
    # set to 1 if forcing_file=BMI
    d['num_timesteps']='1' 
    
    # ---------------------    
    # Options
    # ---------------------
    
    # prints various debug and bmi info
    d['verbosity']='1' 
    
    d['DEBUG']='0'
    
    # Parameter in the surface runoff parameterization (https://mikejohnson51.github.io/hyAggregate/#Routing_Attributes)
    d['refkdt']=f'{row["refkdt"]}'
    
    catchment_configs[row.divide_id] = d
    

In [14]:
base = 'wb-2917533'
for name, conf in catchment_configs.items():
    with open(f'{base}/{name}_config.ini', 'w') as f:
        for k, v in conf.items():
            f.write(f'{k}={v}\n')

In [27]:
class GlobalRealization:
    def __init__(self, global_params=None, time=None, routing=None, catchment_realizations=None):
        self.global_params = global_params
        self.time = time
        self.routing = routing
        self.catchment_realizations = catchment_realizations
    def toJSON(self):
        
        # construct the object that we want to return
        dat = {}
        if self.global_params is not None:
            dat['global'] = self.global_params
        if self.catchment_realizations is not None:
            dat.update(json.loads(catchment_realizations.toJSON()))
        if self.time is not None:
            dat['time'] = self.time
        if self.routing is not None:
            dat['routing'] = self.routing
        
        return json.dumps(dat, sort_keys=False, indent=4)

class CatchmentRealizations:
    def __init__(self):
        self.catchments = {}
    def add_realization(self, name, realization):
        self.catchments[name] = realization
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=False, indent=4)

class Realization:
    def __init__(self, formulation, forcing={}):
        self.formulations = [formulation]
        self.forcing = forcing
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=False, indent=4)

class Formulation:
    def __init__(self, name, params={}, modules=[]):
        self.name = name
        self.params = params
        self.params['modules'] = modules
    def add_module(self, module):
        self.modules.append(module)
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=False, indent=4)
        
class Module:
    def __init__(self, name, library_file, registration_function, params={}):
        self.name = name
        self.library_file = library_file
        self.registration_function = registration_function
        self.params=params
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=False, indent=4)
    

In [28]:
# class Realization:
#     def __init__(self, global_params={}, time={}, routing={}, catchments=[]):
#         self.global_params = global_params
#         self.time = time
#         self.routing = routing
#         self.catchments = []
#     def __add_catchment_realization(self, name, realization):
#         self.catchments[name] = realization
#     def add_catchment_realization(self, name, formulation_modules=[], formulation_params={}):
#         for module in formulation_modules:
#             params = formulation_params
#             if 'modules' not in params:
#                 params['modules'] = []
#             params['modules'].append(module)
#         return params
            
        
#         self.catchments[name] = realization
#     def __create_formulation(self, name, params={}, modules=[]):
#         self.name = name
#         self.params = params
#         self.params['modules'] = modules
#     def add_module(self, module):
#         self.modules.append(module)
        
        
#     def toJSON(self):
#         return json.dumps(self, default=lambda o: o.__dict__, 
#             sort_keys=False, indent=4)

# class CatchmentRealizations:
#     def __init__(self):
#         self.catchments = {}
#     def add_realization(self, name, realization):
#         self.catchments[name] = realization
#     def toJSON(self):
#         return json.dumps(self, default=lambda o: o.__dict__, 
#             sort_keys=False, indent=4)

# class Realization:
#     def __init__(self, formulation, forcing={}):
#         self.formulations = formulation
#         self.forcing = forcing
#     def toJSON(self):
#         return json.dumps(self, default=lambda o: o.__dict__, 
#             sort_keys=False, indent=4)

# class Formulation:
#     def __init__(self, name, params={}, modules=[]):
#         self.name = name
#         self.params = params
#         self.params['modules'] = modules
#     def add_module(self, module):
#         self.modules.append(module)
#     def toJSON(self):
#         return json.dumps(self, default=lambda o: o.__dict__, 
#             sort_keys=False, indent=4)
        
# class Module:
#     def __init__(self, name, library_file, registration_function, params={}):
#         self.name = name
#         self.library_file = library_file
#         self.registration_function = registration_function
#         self.params=params
#     def toJSON(self):
#         return json.dumps(self, default=lambda o: o.__dict__, 
#             sort_keys=False, indent=4)

# def ComplexHandler(Obj):
#     if hasattr(Obj, 'toJSON'):
#         return Obj.toJSON()
#     else:
#         raise TypeError


In [31]:
catchment_realizations = CatchmentRealizations()

for key, val in catchment_configs.items():
    config_name = f'{key}_config.ini'
    forcing_file_path = f'../forcing/{key}.csv'
    
    # CFE
    module_params = {"name": "bmi_c",
                     "model_type_name": "CFE",
                     "main_output_variable": "Q_OUT",
                     "init_config": f"{config_name}",
                     "allow_exceed_end_time": True,
                     "fixed_time_step": False,
                     "uses_forcing_file": True,
                     "forcing_file": forcing_file_path,
                     "variables_names_map": {
                         "atmosphere_water__liquid_equivalent_precipitation_rate": "precip_rate",
                         "water_potential_evaporation_flux": "EVAPOTRANS",
                         "ice_fraction_schaake": "sloth_ice_fraction_schaake",
                         "ice_fraction_xinan": "sloth_ice_fraction_xinan",
                         "soil_moisture_profile": "sloth_smp"
                     }}
    m1 = Module('bmi_c','/dmod/shared_libs/libcfebmi.so.1.0.0','register_bmi_cfe', params=module_params)

    # SLOTH

    module_params = {"name": "bmi_c++",
                     "model_type_name": "SLOTH",
                     "main_output_variable": "z",
                     "init_config": "/dev/null",
                     "allow_exceed_end_time": True,
                     "fixed_time_step": False,
                     "uses_forcing_file": False,
                     "model_params": {
                         "sloth_ice_fraction_schaake(1,double,m,node)": "0.0",
                         "sloth_ice_fraction_xinan(1,double,1,node)": "0.0",
                         "sloth_smp(1,double,1,node)": "0.0",
                         "EVAPOTRANS": "0.0"
                     }}
    m2 = Module('bmi_c++','/dmod/shared_libs/libslothmodel.so','none', params=module_params)


    form_params = {"name": "bmi_multi",
                   "model_type_name": "NoahOWP_CFE",
                   "main_output_variable": "Q_OUT",
                   "init_config": "",
                   "allow_exceed_end_time": False,
                   "fixed_time_step": False,
                   "uses_forcing_file": False}
    f = Formulation('bmi_multi', params=form_params, modules=[m1, m2])

    realization = Realization(f, forcing={'path': forcing_file_path})

    catchment_realizations.add_realization(key, realization)

# create the global realization stuff
time={'start_time': '2010-01-01 00:00:00',
      'end_time'  : '2010-07-01 00:00:00',
      'output_interval': 3600}

routing = {'t_route_config_file_with_path': '/ngen/ngen/data/config/ngen.yaml'}
realization = GlobalRealization(time=time,
                                catchment_realizations=catchment_realizations,
                                routing=routing)

In [32]:
with open(f'{base}/realization.json', 'w') as f:
    f.write(realization.toJSON())

In [54]:
import cfe_realization as r
from pathlib import Path

In [66]:
import importlib
importlib.reload(r)

<module 'cfe_realization' from '/home/jovyan/Desktop/notebook-examples/ngen/cfe_realization.py'>

In [58]:
base = Path('wb-2917533')
cfe_atts_path = Path('wb-2917533/cfe_noahowp_attributes.csv')

In [59]:
r.make_catchment_configs(base, cfe_atts_path)

In [67]:
time={'start_time': '2010-01-01 00:00:00',
      'end_time'  : '2010-07-01 00:00:00',
      'output_interval': 3600}
r.create_cfe_realization(base, cfe_atts_path, time=time)

In [69]:
!rm -rf wb-2917533 cat-679069 cat-680662/